# Quantum Process Tomography via Genetic Algorithms

## Experimental Transformation
___

This notebook aims at reproducing the experiments proposed in Section V. of 'Retrieving complex polarization transformations via optimized quantum process tomography', paper submitted to ... .

By means of this code you can reconstruct the images available in the git repository.

You can easly use the code for your problem, by modelling the measured data in a similar way to the data available in the repository. 

________

Let us start by importing the required libraries:

In [ ]:
from deap import base
from deap import creator
from deap import tools
import numpy as np
import random
import math
import pandas as pd
import time
import itertools
import utils.GA_utils as GA_utils
import statistics
import seaborn as sns
import matplotlib.pyplot as plt

Then, the genetic reconstruction of the whole experimental image can start. 

Firstily, we define the fitness function of the problem.

In [ ]:
def evaluate(HH, LL, HL, LD, LH, HD, individual):
    Energy, nx, ny, nz = individual[0], individual[1], individual[2], individual[3]
    f = ((math.cos(Energy)**2 - HH + nx**2 * math.sin(Energy)**2)**2) / HH + ((math.cos(Energy)**2 - LL + nz**2 * math.sin(Energy)**2)**2) / LL + ( ( -HL + 1/2 * (math.cos(Energy)**2 - 2 * ny * math.cos(Energy) * math.sin(Energy) + (ny**2 + (nx + nz)**2) * math.sin(Energy)**2))**2 )/ HL + ( ( -LD + 1/2 * (math.cos(Energy)**2 - 2 * nx * math.cos(Energy) * math.sin(Energy) + (nx**2 + (ny + nz)**2) * math.sin(Energy)**2))**2 ) / LD + ((-LH + 1/2 * (math.cos(Energy)**2 + (ny**2 + (nx + nz)**2) * math.sin(Energy)**2 + ny * math.sin(2 * Energy)))**2 )/ LH + (( -HD + 1/2 * (math.cos(Energy)**2 + ((nx + ny)**2 + nz**2) * math.sin(Energy)**2 +nz * math.sin(2 * Energy)))**2)/ HD
    return f

Secondly, we define the hyperparameter of the GA. 

The parameters below refer to those reported in the paper. 

In [ ]:
POP_SIZE = 30
CXPB = 0.8
MUTPB = 0.1
NGEN = 100
STATS = GA_utils.createStats()
pop_list = None
TS_SIZE = 3

The function `compute_unitary` will be used for computing the unitary $U$, given $\Theta\in[0,\pi]$ and $\mathbf{n}=(n_x,n_y,n_z)$ according to:

\begin{equation}
U=\begin{pmatrix}
\cos \Theta -i \sin \Theta \,n_z && -i\sin \Theta \,(n_x-i n_y)\\
-i\sin \Theta \,(n_x+i n_y) && \cos \Theta + i \sin \Theta \,n_z
\end{pmatrix}
\end{equation}

In [ ]:
def compute_unitary(Theta, nx, ny, nz):
    I = np.array([[1, 0], [0, 1]])
    sx = np.matrix([[0, 1], [1, 0]])
    sy = np.matrix([[0, -1j], [1j, 0]])
    sz = np.matrix([[1, 0], [0, -1]])
    return math.cos(Theta) * I - 1j * math.sin(Theta) * (nx * sx + ny * sy + nz * sz)

By running the following cell you can import the experimental data.

Please select a problem from 1 to 3, for replicating the experiment reported in the paper. 

In [ ]:
problem = "3"
path = "experimental_data_73/"
LL = np.loadtxt(path + problem + "/LL.txt", dtype="f", delimiter="\t")
DL = np.loadtxt(path + problem + "/DL.txt", dtype="f", delimiter="\t")
HH = np.loadtxt(path + problem + "/HH.txt", dtype="f", delimiter="\t")
HL = np.loadtxt(path + problem + "/HL.txt", dtype="f", delimiter="\t")
LD = np.loadtxt(path + problem + "/LD.txt", dtype="f", delimiter="\t")
LH = np.loadtxt(path + problem + "/LH.txt", dtype="f", delimiter="\t")
HD = np.loadtxt(path + problem + "/HD.txt", dtype="f", delimiter="\t")
print(LL.shape, HH.shape, HL.shape, LD.shape, LH.shape, HD.shape)

In [ ]:
theta_t = np.loadtxt(path+problem+"/thetath.txt", dtype='f', delimiter='\t')
nx_t = np.loadtxt(path+problem+"/nxth.txt", dtype='f', delimiter='\t')
ny_t = np.loadtxt(path+problem+"/nyth.txt", dtype='f', delimiter='\t')
nz_t = np.loadtxt(path+problem+"/nzth.txt", dtype='f', delimiter='\t')

The following cells start the reconstruction pixel by pixel of the image. 

In [ ]:
im_size_x, im_size_y = 73, 73

In [ ]:
toolbox = GA_utils.createToolbox()
toolbox.decorate("mutate", GA_utils.checkBounds(0, np.pi))
toolbox.decorate("mate", GA_utils.checkBounds(0, np.pi))

final_pop_pixels = {
    str(pixel[0]) + "_" + str(pixel[1]): ""
    for pixel in itertools.product(
        [i for i in range(im_size_x)], [j for j in range(im_size_y)]
    )
}
best_ind_pixels = {
    str(pixel[0]) + "_" + str(pixel[1]): ""
    for pixel in itertools.product(
        [i for i in range(im_size_x)], [j for j in range(im_size_y)]
    )
}
start_time = time.time()
for i in range(im_size_x):
    for j in range(im_size_y):
        toolbox.register(
            "evaluate",
            evaluate,
            HH[i][j],
            LL[i][j],
            HL[i][j],
            LD[i][j],
            LH[i][j],
            HD[i][j],
        )

        if i == 0 and j == 0:
            s = True
            NGEN = NGEN
        else:
            s = False
            NGEN = 10

        GA = GA_utils.updateGA_Map(
            toolbox,
            pixel=(i, j),
            im_size_x=im_size_x,
            im_size_y=im_size_y,
            best_ind_pixels=best_ind_pixels,
            pop_size=POP_SIZE,
            cxpb=CXPB,
            mutpb=MUTPB,
            ngen=NGEN,
            stats=STATS,
            tourn_size=TS_SIZE,
            hof=tools.HallOfFame(1),
            starting_point=s,
            verbose=False,
        )
        (
            final_pop_pixels[str(i) + "_" + str(j)],
            best_ind_pixels[str(i) + "_" + str(j)],
        ) = (GA[0], GA[2][0])
print("Reconstruction time in s: ", time.time() - start_time)

By executing the following cell you can computing the fidelities of the reconstructed processes

In [ ]:
fidelities = {}
im_theta, im_theta_t = {},{}
im_nx, im_nx_t = {},{}
im_ny, im_ny_t = {},{}
im_nz, im_nz_t = {},{}

for i in range(im_size_x):
    for j in range(im_size_y):
        individual = best_ind_pixels[str(i)+'_'+str(j)]
        theta = individual[0]
        nx = individual[1]
        ny = individual[2]
        nz = individual[3]
        im_theta[str(i)+'_'+str(j)], im_nx[str(i)+'_'+str(j)], im_ny[str(i)+'_'+str(j)], im_nz[str(i)+'_'+str(j)]= theta,nx,ny,nz

        computed_u = compute_unitary(theta, nx, ny, nz)
        data_u = compute_unitary(theta_t[i][j],nx_t[i][j],ny_t[i][j],nz_t[i][j])
        im_theta_t[str(i)+'_'+str(j)], im_nx_t[str(i)+'_'+str(j)], im_ny_t[str(i)+'_'+str(j)], im_nz_t[str(i)+'_'+str(j)]= theta_t[i][j],nx_t[i][j],ny_t[i][j],nz_t[i][j]
        F = 0.5 * np.linalg.norm(np.trace(data_u.getH() * computed_u))
        fidelities[str(i)+'_'+str(j)]=F

The mean fidelity is computed in the following cell:

In [ ]:
print(statistics.mean(list(fidelities.values())))

Let's graphically analyze the results.

#### Theoretical Theta 

In [ ]:
im_theta_np_t = np.array(list(im_theta_t.values()))
tehta_t_ = np.reshape(im_theta_np_t, (im_size_x, im_size_y))


plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    theta_t, cmap="YlGnBu", xticklabels=False, yticklabels=False, vmin=0, vmax=math.pi
)
ax.set_xlabel(r"$q_x$", fontsize=14)
ax.set_ylabel(r"$q_y$", fontsize=14)
ax.set_xticks([0, 34])
ax.set_xticklabels(["0", r"2$\pi$"])
ax.set_yticks([0])
ax.set_yticklabels([r"2$\pi$"])
cbar = ax.collections[0].colorbar
cbar.set_ticks([0, math.pi])
cbar.set_ticklabels(["0", r"$\pi$"])
plt.tight_layout()
plt.show()

#### Reconstructed Theta

In [ ]:
im_theta_np = np.array(list(im_theta.values()))
theta_ = np.reshape(im_theta_np, (im_size_x, im_size_y))


plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    theta_,
    cmap="YlGnBu",
    xticklabels=False,
    yticklabels=False,
    cbar=False,
    vmin=0,
    vmax=math.pi,
)
ax.set_xlabel(r"$q_x$", fontsize=14)
ax.set_ylabel(r"$q_y$", fontsize=14)
ax.set_xticks([0, 34])
ax.set_xticklabels(["0", r"2$\pi$"])
ax.set_yticks([0])
ax.set_yticklabels([r"2$\pi$"])

plt.tight_layout()

#### Theoretical $N_X$

In [ ]:
im_nx_np_t = np.array(list(im_nx_t.values()))
nx_t_ = np.reshape(im_nx_np_t, (im_size_x, im_size_y))

plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    nx_t, cmap="YlGnBu", xticklabels=False, yticklabels=False, vmin=-1, vmax=1
)
ax.set_xlabel(r"$q_x$", fontsize=14)
ax.set_ylabel(r"$q_y$", fontsize=14)
ax.set_xticks([0, 34])
ax.set_xticklabels(["0", r"2$\pi$"])
ax.set_yticks([0])
ax.set_yticklabels([r"2$\pi$"])
cbar = ax.collections[0].colorbar
cbar.set_ticks([-1, 1])
cbar.set_ticklabels(["-1", "1"])
plt.tight_layout()

#### Reconstructed $n_x$

In [ ]:
im_nx_np = np.array(list(im_nx.values()))
nx_ = np.reshape(-im_nx_np, (im_size_x, im_size_y))

plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    -nx_,
    cmap="YlGnBu",
    xticklabels=False,
    yticklabels=False,
    cbar=False,
    vmin=-1,
    vmax=1,
)
ax.set_xlabel(r"$q_x$", fontsize=14)
ax.set_ylabel(r"$q_y$", fontsize=14)
ax.set_xticks([0, 34])
ax.set_xticklabels(["0", r"2$\pi$"])
ax.set_yticks([0])
ax.set_yticklabels([r"2$\pi$"])
plt.tight_layout()

#### Theoretical $N_y$

In [ ]:
fig = plt.plot()
im_ny_np_t = np.array(list(im_ny_t.values()))
ny_t = np.reshape(im_ny_np_t, (im_size_x, im_size_y))


plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    ny_t, cmap="YlGnBu", xticklabels=False, yticklabels=False, vmin=-1, vmax=1
)
ax.set_xlabel(r"$q_x$", fontsize=14)
ax.set_ylabel(r"$q_y$", fontsize=14)
ax.set_xticks([0, 34])
ax.set_xticklabels(["0", r"2$\pi$"])
ax.set_yticks([0])
ax.set_yticklabels([r"2$\pi$"])
cbar = ax.collections[0].colorbar
cbar.set_ticks([-1, 1])
cbar.set_ticklabels(["-1", "1"])
plt.tight_layout()

#### Reconstructed $N_y$

In [ ]:
fig = plt.plot()
im_ny_np = np.array(list(im_ny.values()))
ny_ = np.reshape(im_ny_np, (im_size_x, im_size_y))

plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    ny_,
    cmap="YlGnBu",
    xticklabels=False,
    yticklabels=False,
    cbar=False,
    vmin=-1,
    vmax=1,
)
ax.set_xlabel(r"$q_x$", fontsize=14)
ax.set_ylabel(r"$q_y$", fontsize=14)
ax.set_xticks([0, 34])
ax.set_xticklabels(["0", r"2$\pi$"])
ax.set_yticks([0])
ax.set_yticklabels([r"2$\pi$"])
plt.tight_layout()

#### Theoretical $N_z$

In [ ]:
im_nz_np_t = np.array(list(im_nz_t.values()))
nz_t_ = np.reshape(im_nz_np_t, (im_size_x, im_size_y))

plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    nz_t, cmap="YlGnBu", xticklabels=False, yticklabels=False, vmin=-1, vmax=1
)
ax.set_xlabel(r"$q_x$", fontsize=14)
ax.set_ylabel(r"$q_y$", fontsize=14)
ax.set_xticks([0, 34])
ax.set_xticklabels(["0", r"2$\pi$"])
ax.set_yticks([0])
ax.set_yticklabels([r"2$\pi$"])
cbar = ax.collections[0].colorbar
cbar.set_ticks([-1, 1])
cbar.set_ticklabels(["-1", "1"])
plt.tight_layout()

#### Reconstructed $N_z$

In [ ]:
im_nz_np = np.array(list(im_nz.values()))
nz_ = np.reshape(im_nz_np, (im_size_x, im_size_y))


plt.rcParams["figure.figsize"] = (8, 6)
ax = sns.heatmap(
    nz_,
    cmap="YlGnBu",
    xticklabels=False,
    yticklabels=False,
    cbar=False,
    vmin=-1,
    vmax=1,
)
ax.set_xlabel(r"$q_x$", fontsize=14)
ax.set_ylabel(r"$q_y$", fontsize=14)
ax.set_xticks([0, 34])
ax.set_xticklabels(["0", r"2$\pi$"])
ax.set_yticks([0])
ax.set_yticklabels([r"2$\pi$"])
plt.tight_layout()